In [1]:
import pandas as pd
# import sklearn
# from sklearn.feature_selection import RFE
# from sklearn.preprocessing import StandardScaler, MinMaxScaler
# # from sklearn.linear_model import Ridge
# from sklearn.preprocessing import PolynomialFeatures, OneHotEncoder
# # from sklearn.linear_model import LinearRegression

# import tensorflow as tf
# import numpy as np
# from sklearn.metrics import mean_squared_error as mse
# from sklearn.model_selection import train_test_split
# from sklearn.utils import resample
# from sklearn.metrics import mean_squared_log_error


# import matplotlib 

In [2]:
holiday_events_df = pd.read_csv("../raw_data/holidays_events.csv")
oil_df = pd.read_csv("../raw_data/oil.csv")
#sample_submission_df = pd.read_csv("../raw_data/sample_submission.csv")
#stores_df = pd.read_csv("../raw_data/stores.csv")
test_df = pd.read_csv("../raw_data/test.csv")
train_df = pd.read_csv("../raw_data/train.csv")
#transactions_df = pd.read_csv("../raw_data/transactions.csv")

In [3]:
train_df['date']=pd.to_datetime(train_df['date'])
test_df['date']=pd.to_datetime(test_df['date'])
#transactions_df['date']=pd.to_datetime(transactions_df['date'])
holiday_events_df['date']=pd.to_datetime(holiday_events_df['date'])
oil_df['date']=pd.to_datetime(oil_df['date'])

In [4]:
holiday_events_df.drop(columns=['description', 'transferred'], inplace=True)


In [5]:
oil_df.rename(columns={'dcoilwtico': 'oil_price'}, inplace=True)
oil_df.dropna(subset=['oil_price'], inplace=True)


In [6]:
holiday_oil_df = pd.merge(holiday_events_df, oil_df, on='date')

In [7]:
train_df.drop(columns=['id'],inplace=True)
test_df.drop(columns=['id'],inplace=True)

In [8]:
train_joined = pd.merge(train_df, holiday_oil_df[['date','type','locale','locale_name','oil_price']].groupby('date').agg({'type':' '.join, 'locale':' '.join, 'locale_name':' '.join, 'oil_price':'mean'}), 
                      left_on=['date'], right_on=['date'], how='left')
train_joined['oil_price'].fillna(train_joined['oil_price'].mean(), inplace=True)

In [9]:
train_joined

,date,store_nbr,family,sales,onpromotion,type,locale,locale_name,oil_price
0,2013-01-01,1,AUTOMOTIVE,0.000,0,NaN,NaN,NaN,64.497892
1,2013-01-01,1,BABY CARE,0.000,0,NaN,NaN,NaN,64.497892
2,2013-01-01,1,BEAUTY,0.000,0,NaN,NaN,NaN,64.497892
3,2013-01-01,1,BEVERAGES,0.000,0,NaN,NaN,NaN,64.497892
4,2013-01-01,1,BOOKS,0.000,0,NaN,NaN,NaN,64.497892
...,...,...,...,...,...,...,...,...,...
3000883,2017-08-15,9,POULTRY,438.133,0,Holiday,Local,Riobamba,47.570000
3000884,2017-08-15,9,PREPARED FOODS,154.553,1,Holiday,Local,Riobamba,47.570000
3000885,2017-08-15,9,PRODUCE,2419.729,148,Holiday,Local,Riobamba,47.570000
3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Holiday,Local,Riobamba,47.570000


In [10]:
test_joined = pd.merge(test_df, holiday_oil_df[['date','type','locale','locale_name','oil_price']].groupby('date').agg({'type':' '.join, 'locale':' '.join, 'locale_name':' '.join, 'oil_price':'mean'}), 
                      left_on=['date'], right_on=['date'], how='left')
test_joined['oil_price'].fillna(train_joined['oil_price'].mean(), inplace=True)

In [11]:
test_joined


,date,store_nbr,family,onpromotion,type,locale,locale_name,oil_price
0,2017-08-16,1,AUTOMOTIVE,0,NaN,NaN,NaN,64.497892
1,2017-08-16,1,BABY CARE,0,NaN,NaN,NaN,64.497892
2,2017-08-16,1,BEAUTY,2,NaN,NaN,NaN,64.497892
3,2017-08-16,1,BEVERAGES,20,NaN,NaN,NaN,64.497892
4,2017-08-16,1,BOOKS,0,NaN,NaN,NaN,64.497892
...,...,...,...,...,...,...,...,...
28507,2017-08-31,9,POULTRY,1,NaN,NaN,NaN,64.497892
28508,2017-08-31,9,PREPARED FOODS,0,NaN,NaN,NaN,64.497892
28509,2017-08-31,9,PRODUCE,1,NaN,NaN,NaN,64.497892
28510,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9,NaN,NaN,NaN,64.497892


## Group by date

In [12]:

def get_mode(series):
    mode_val = series.mode().values[0] if not series.isnull().all() else series.mode().values[0] if not series.dropna().empty else None
    return mode_val
# Group by 'date' and apply the 'get_mode()' function to each column
train_by_date = train_joined.groupby('date').agg({'type': get_mode, 'locale': get_mode, 'locale_name': get_mode, 
                                                   'onpromotion': 'sum', 'oil_price': 'mean', 'sales': 'sum'})
train_by_date.columns = ['type', 'locale', 'locale_name', 'onpromotion', 'oil_price', 'sales']
train_by_date.reset_index(inplace=True)



In [13]:
train_by_date

,date,type,locale,locale_name,onpromotion,oil_price,sales
0,2013-01-01,None,None,None,0,64.497892,2511.618999
1,2013-01-02,None,None,None,0,64.497892,496092.417944
2,2013-01-03,None,None,None,0,64.497892,361461.231124
3,2013-01-04,None,None,None,0,64.497892,354459.677093
4,2013-01-05,None,None,None,0,64.497892,477350.121229
...,...,...,...,...,...,...,...
1679,2017-08-11,Transfer,National,Ecuador,14179,48.810000,826373.722022
1680,2017-08-12,None,None,None,8312,64.497892,792630.535079
1681,2017-08-13,None,None,None,9283,64.497892,865639.677471
1682,2017-08-14,None,None,None,8043,64.497892,760922.406081


In [14]:
# Group by 'date' and apply the 'get_mode()' function to each column
test_by_date = test_joined.groupby('date').agg({'type': get_mode, 'locale': get_mode, 
                                                'locale_name': get_mode, 'onpromotion': 'sum', 
                                                'oil_price': get_mode})
test_by_date.columns = ['type', 'locale', 'locale_name', 'onpromotion', 'oil_price']
test_by_date.reset_index(inplace=True)

In [15]:
test_by_date

,date,type,locale,locale_name,onpromotion,oil_price
0,2017-08-16,None,None,None,30539,64.497892
1,2017-08-17,None,None,None,7505,64.497892
2,2017-08-18,None,None,None,13777,64.497892
3,2017-08-19,None,None,None,8614,64.497892
4,2017-08-20,None,None,None,9513,64.497892
5,2017-08-21,None,None,None,8100,64.497892
6,2017-08-22,None,None,None,10295,64.497892
7,2017-08-23,None,None,None,16500,64.497892
8,2017-08-24,Holiday,Local,Ambato,8268,47.240000
9,2017-08-25,None,None,None,13991,64.497892


## Group by store_nbr

In [16]:
# for train data: group by 'store_nbr' and apply the 'get_mode()' function to each column
train_by_store = train_joined.groupby('store_nbr').agg({'sales': 'sum', 'onpromotion': 'sum'})
train_by_store.columns = ['onpromotion', 'sales']
train_by_store.reset_index(inplace=True)



In [17]:
train_by_store

,store_nbr,onpromotion,sales
0,1,1.414501e+07,138020
1,2,2.155739e+07,158149
2,3,5.048191e+07,177075
3,4,1.890970e+07,151932
4,5,1.559240e+07,149370
5,6,2.518368e+07,169761
6,7,2.695205e+07,153875
7,8,3.049429e+07,169671
8,9,2.640944e+07,177356
9,10,9.613906e+06,103223


In [18]:
# For test data, group by 'date' and apply the 'get_mode()' function to each column
test_by_store = test_joined.groupby('store_nbr').agg({'onpromotion': 'sum'})
test_by_store.columns = ['onpromotion']
test_by_store.reset_index(inplace=True)

In [19]:
test_by_store

,store_nbr,onpromotion
0,1,3142
1,2,3599
2,3,3995
3,4,3391
4,5,3253
5,6,3704
6,7,3475
7,8,3487
8,9,4396
9,10,3976
